## Service Dependency Map using Graph Theory

I've been working on this intriguing project to visualize network connections using Python and Dash. By executing the netstat command, I gather data on established connections and then map these onto a network graph using NetworkX. The graph highlights the interconnectedness of services, revealing how different nodes (IP addresses) communicate.

The benefits are clear: enhanced network visibility and troubleshooting. It’s easier to spot anomalies or points of failure. Moving forward, I plan to integrate real-time data for dynamic network analysis and introduce machine learning for predictive insights, which could revolutionize network management and security.

### Import libraries

In [1]:
import subprocess
import dash
from dash import dcc, html
import networkx as nx
import plotly.graph_objs as go

### Extracting and Processing Network Connection Data with Python
The code executes the 'netstat' command to retrieve network connection details, then decodes and splits the output to analyze each line separately. This is a foundational step in gathering network status information for further processing or visualization

In [4]:
# Execute the netstat command
#output = subprocess.check_output(['netstat', '-an']).decode('utf-8').split("\n")
output = subprocess.check_output('netstat -an', shell=True).decode('utf-8').split("\n")

### Building a Network Graph from Connection Data
The code constructs a network graph using NetworkX, adding nodes and edges from a list of established connections (denoted by "ESTABLISHED"). Each line of the input is parsed to extract source and target IP addresses, which are added as nodes, and a connection between them is represented as an edge. The graph layout is determined using the spring layout algorithm, arranging the nodes based on their connections

In [9]:
# Initialize a network graph
G = nx.Graph()

for line in output:
    if "ESTABLISHED" not in line:
        continue

    parts = line.split()
    # In Windows, the IP and port are typically in the 2nd and 3rd elements
    source = parts[1]
    target = parts[2]

    G.add_node(source)
    G.add_node(target)
    G.add_edge(source, target)

# Get positions for the nodes in G
pos = nx.spring_layout(G)

### Constructing Network Edge Visualizations in Dash
This code segment creates visual representations of network edges for a Dash application. It iterates through each edge in a network graph, extracts the coordinates for the connected nodes, and stores them for plotting. The go.Scatter function is then used to draw these edges as lines on a Dash visualization.

In [10]:
# Create edge traces for Dash visualization
edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

### Developing an Interactive Network Visualization Dashboard
This code constructs an interactive dashboard using Dash for visualizing network nodes. It creates a scatter plot (node trace) with node positions and IP addresses for display, and then integrates this into a Dash application layout, showcasing a "Service Dependency Map". The visualization emphasizes the connections between various nodes in a network, offering a user-friendly interface for exploring network structures.

In [11]:
# Create node traces for Dash visualization
node_x = []
node_y = []
text = []
for node in pos:
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    text.append(node)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
        ),
        line=dict(width=2)),
    text=text  # this line will show IP address
)

# Build the Dash app
app = dash.Dash(__name__)
app.layout = html.Div([
    dcc.Graph(
        id="service-dependency-map",
        figure={
            'data': [edge_trace, node_trace],
            'layout': go.Layout(
                title='Service Dependency Map',
                showlegend=False,
                hovermode='closest',
                margin=dict(b=0, l=0, r=0, t=40),
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
        }
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)